# Il più semplice esempio di interazione con modelli locali via l'API di OpenAI

Luca Mari, gennaio 2025  

Quest'opera è distribuita con <a href="http://creativecommons.org/licenses/by-nc-sa/4.0" target="_blank">Licenza Creative Commons Attribuzione - Non commerciale - Condividi allo stesso modo 4.0 Internazionale</a>.  
<img src="https://creativecommons.it/chapterIT/wp-content/uploads/2021/01/by-nc-sa.eu_.png" width="100">

**Obiettivo**: comprendere le più semplice modalità di interazione con modelli locali via l'API di OpenAI.  
**Precompetenze**: basi di Python.

> Per eseguire questo notebook con VSCode sul proprio calcolatore, occorre:
> * installare un interprete Python
> * attivare un server locale che renda possibile l'interazione con un modello via l'API di OpenAI (per semplicità, supporremo che sia LM Studio, scaricabile da https://lmstudio.ai)
> * scaricare da https://code.visualstudio.com/download e installare VSCode
> * eseguire VSCode e attivare le estensioni per Python e Jupyter
> * ancora in VSCode:
>     * creare una cartella di lavoro e renderla la cartella corrente
>     * copiare nella cartella il file di questa attività: [oaibase.ipynb](oaibase.ipynb)
>     * aprire il notebook `oaibase.ipynb`
>     * creare un ambiente virtuale locale Python (Select Kernel | Python Environments | Create Python Environment | Venv, e scegliere un interprete Python):
>     * installare i moduli Python richiesti, eseguendo dal terminale:  
>         `pip install openai`

Importiamo i moduli Python necessari e specifichiamo l'_end point_ per l'accesso al server locale.

In [10]:
from openai import OpenAI
from pprint import pprint
from IPython.display import Markdown, display
import json

def stream_print(response, max_length=100):
    length = 0
    for chunk in response:
        text = chunk.choices[0].delta
        if hasattr(text, 'content') and text.content:
            print(text.content, end='', flush=True)
            length += len(text.content)
            if length > max_length:
                print()
                length = 0


client = OpenAI(base_url="http://localhost:1234/v1") # usa un server locale, per esempio con LM Studio  

### Elenco dei modelli disponibili
Assumendo che il server sia attivo, questo è il più semplice esempio di una richiesta al server via l'API di OpenAI, per ottenere la lista dei modelli accessibili attraverso il server stesso (ma in effetti dunque anche per accertare che il server sia accessibile).

In [9]:
models = client.models.list()
pprint(models.data)

[Model(id='gemma-3-4b-it', created=None, object='model', owned_by='organization_owner'),
 Model(id='gemma-3-27b-it', created=None, object='model', owned_by='organization_owner'),
 Model(id='text-embedding-nomic-embed-text-v1.5@q4_k_m', created=None, object='model', owned_by='organization_owner'),
 Model(id='qwen2.5-32b-instruct', created=None, object='model', owned_by='organization_owner'),
 Model(id='qwq-32b', created=None, object='model', owned_by='organization_owner'),
 Model(id='simplescaling_s1.1-32b', created=None, object='model', owned_by='organization_owner'),
 Model(id='openthinker-32b', created=None, object='model', owned_by='organization_owner'),
 Model(id='deepseek-r1-distill-qwen-32b', created=None, object='model', owned_by='organization_owner'),
 Model(id='deepseek-r1-distill-qwen-1.5b', created=None, object='model', owned_by='organization_owner'),
 Model(id='deepseek-r1-distill-llama-8b', created=None, object='model', owned_by='organization_owner'),
 Model(id='deepseek-r

### Generazione di sentence embedding
Assumendo che sia stato caricato un modello di _embedding_ sul server, possiamo usarlo per fare _sentence embedding_. 

In [22]:
text = "La bellezza salverà il mondo."

response = client.embeddings.create(
    model="...",
    input=[text]
).data[0].embedding

print(f'''La frase è stata codificata in un vettore di {len(response)} numeri;
i primi 5 sono: {response[:5]}''')

La frase è stata codificata in un vettore di 768 numeri;
i primi 5 sono: [0.10395540297031403, 0.050489071756601334, -0.0963636264204979, 0.011676624417304993, -0.04801320657134056]


### Completion
Questo è il più semplice esempio di una richiesta al server via l'API di OpenAI (stiamo supponendo che il server sia attivo e sia stato caricato un modello).

In [3]:
prompt = "Presentati: chi sei?"

response = client.chat.completions.create(
    model="...",
    messages=[
        {"role": "system", "content": "Rispondi sempre in italiano."},
        {"role": "user", "content": prompt}
    ],
    max_tokens=-1,
    temperature=0.7,
    stream=False
)

print("L'intero oggetto JSON di risposta:")
pprint(dict(response))

print("\nIl messaggio generato:")
display(Markdown(response.choices[0].message.content))

L'intero oggetto JSON di risposta:
{'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Ciao! Sono Gemma, un modello linguistico di grandi dimensioni creato dal team di Google DeepMind. Sono un'intelligenza artificiale open-weights, il che significa che sono ampiamente disponibile al pubblico. Posso aiutarti con una varietà di compiti che riguardano il linguaggio, come rispondere a domande, generare testo creativo e tradurre lingue. \n\nCosa posso fare per te oggi?\n", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))],
 'created': 1741886738,
 'id': 'chatcmpl-9rf1hsgh6ifyjroq5p5mm',
 'model': 'gemma-3-4b-it',
 'object': 'chat.completion',
 'service_tier': None,
 'stats': {},
 'system_fingerprint': 'gemma-3-4b-it',
 'usage': CompletionUsage(completion_tokens=84, prompt_tokens=23, total_tokens=107, completion_tokens_details=None, prompt_tokens_details=None)}

Il messaggio generato:


Ciao! Sono Gemma, un modello linguistico di grandi dimensioni creato dal team di Google DeepMind. Sono un'intelligenza artificiale open-weights, il che significa che sono ampiamente disponibile al pubblico. Posso aiutarti con una varietà di compiti che riguardano il linguaggio, come rispondere a domande, generare testo creativo e tradurre lingue. 

Cosa posso fare per te oggi?


### Completion con stream dei token
Quasi identica all'esempio precedente è la richiesta di una risposta che sia inviata un token per volta.

In [11]:
prompt = "Presentati: chi sei?"

response = client.chat.completions.create(
    model="...",
    messages=[
        {"role": "system", "content": "Rispondi sempre in italiano."},
        {"role": "user", "content": prompt}
    ],
    max_tokens=-1,
    temperature=0.7,
    stream=True
)

stream_print(response)

Ciao! Sono Gemma, un modello linguistico di grandi dimensioni creato dal team di Google DeepMind. Sono
 un'intelligenza artificiale progettata per rispondere alle tue domande e generare testo creativo. Sono
 un modello a "pesi aperti", quindi sono disponibile pubblicamente. 

Cosa ti incuriosisce?


### Completion con structured output
Solo un poco più complessa è una richiesta in cui si specifica lo schema JSON che vogliamo sia utilizzato nella risposta, nuovamente in accordo all'API di OpenAI, nella specifica _structured output_ (https://platform.openai.com/docs/guides/structured-outputs).

In [ ]:
response_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "presentazione",
        "strict": "true",
        "schema": {
            "type": "object",
            "properties": {
                "nome": {
                    "type": "string",
                    "description": "Il tuo nome"
                },
                "produttore": {
                    "type": "string",
                    "description": "Il nome del tuo produttore",
                    "enum": ["OpenAI", "Google", "Meta", "other"]
                },
                "caratteristiche principali": {
                    "type": "string",
                    "description": "Le tue caratteristiche principali"
                },
                "caratteristiche secondarie": {
                    "type": "string",
                    "description": "Le tue caratteristiche secondarie (opzionale)"
                }
            },
        "required": ["nome", "produttore", "caratteristiche principali"]
        }
    }
}

prompt = "Presentati: chi sei?"

response = client.chat.completions.create(
    model="...",
    messages=[
        {"role": "system", "content": "Rispondi sempre in italiano." },
        {"role": "user", "content": prompt }
    ],
    max_tokens=-1,
    temperature=0.7,
    stream=True,
    response_format=response_format # type: ignore
)

stream_print(response)

{
  "nome": "Gemini",
  "produttore": "Google",
  "caratteristiche principali": "Sono un modello lingu
istico di grandi dimensioni, addestrato da Google. Posso comunicare e generare testo simile a quello umano
 in risposta a una vasta gamma di prompt e domande. Ad esempio, posso fornire riassunti di argomenti fatt
uali o creare storie."
}
 	 	 	 	 	 	 	 	 	 	

### Function calling

https://platform.openai.com/docs/guides/function-calling 

In [ ]:
def compute_sqrt(number: float) -> float:   # la funzione da chiamare
    return number ** 0.5

tools = [                                   # la definizione della funzione
    {
        "type": "function",
        "function": {
            "name": "compute_sqrt",
            "description": "Calcola la radice quadrata di un numero",
            "parameters": {
                "type": "object",
                "properties": {
                    "number": {
                        "type": "float",
                        "description": "Il numero di cui calcolare la radice quadrata"
                    }
                },
                "required": ["number"]
            }
        }
    }
]

prompt = "Calcola la radice quadrata di 64."

response = client.chat.completions.create(
    model="...",
    messages=[
        {"role": "system", "content": "Rispondi sempre in italiano." },
        {"role": "user", "content": prompt }
    ],
    max_tokens=-1,
    temperature=0.7,
    tools=tools # type: ignore
)

In [16]:
print("L'intero oggetto JSON di risposta:")
pprint(dict(response))

print("\nIl messaggio generato:")
pprint(response.choices[0].message)

if response.choices[0].message.tool_calls:
    tool_call = response.choices[0].message.tool_calls[0]

    print("\nLa parte del messaggio riferita alla funzione da chiamare:")
    print(tool_call.function)

    function_name = tool_call.function.name
    function_arguments = json.loads(tool_call.function.arguments)
    print(f"\nLa funzione da chiamare è: {function_name}; i suoi argomenti sono: {function_arguments}")

L'intero oggetto JSON di risposta:
{'choices': [Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='891526812', function=Function(arguments='{"number":64}', name='compute_sqrt'), type='function')]))],
 'created': 1741888794,
 'id': 'chatcmpl-umb4jvw0s4n3u00801kq',
 'model': 'gemma-3-4b-it',
 'object': 'chat.completion',
 'service_tier': None,
 'stats': {},
 'system_fingerprint': 'gemma-3-4b-it',
 'usage': CompletionUsage(completion_tokens=35, prompt_tokens=423, total_tokens=458, completion_tokens_details=None, prompt_tokens_details=None)}

Il messaggio generato:
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='891526812', function=Function(arguments='{"number":64}', name='compute_sqrt'), type='fun

In [17]:
to_evaluate = function_name + "(" + str(function_arguments['number']) + ")"
result = eval(to_evaluate)

print(f"\nIl risultato della funzione è: {result}")


Il risultato della funzione è: 8.0
